<a href="https://colab.research.google.com/github/francescagiacco/feminicides-italy/blob/main/scripts/03_test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TESTING THE PRE-TRAINED MODEL(S)**

In this script I test some pre-trained models on my data. The goal is to get a victim blaming score and assess if it makes sense. 

For the analysis in my thesis I end up using **MODEL 2 at the sentence level** : https://huggingface.co/responsibility-framing/predict-perception-bert-blame-victim  which can be found scrolling down the script


**Other Resources**


*   AutoClasses: https://huggingface.co/docs/transformers/v4.27.2/en/model_doc/auto#transformers.AutoModel
*   BERT: https://huggingface.co/docs/transformers/main/en/model_doc/bert#transformers.BertForMaskedLM




In [ ]:
#install necessary packages 
!pip install transformers
#!pip install transformers_interpret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.2 MB/s eta 0:00:00


In [ ]:
#load packages
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers.pipelines.text_classification import ClassificationFunction


In [ ]:
#read dataset
datac= pd.read_csv("/content/drive/MyDrive/thesis/labeltest.csv")


<ipython-input-5-70c0c6f15d7c>:1: DtypeWarning: Columns (14,27,48) have mixed types. Specify dtype option on import or set low_memory=False.
  datac= pd.read_csv("/content/drive/MyDrive/thesis/labeltest.csv")


**Model 1: Blame on the assassin**

In [ ]:
load model - blame assassin
tokenizer1 = AutoTokenizer.from_pretrained("responsibility-framing/predict-perception-bert-blame-assassin", max_length=512, truncation=True, padding=True)

model1 = AutoModelForSequenceClassification.from_pretrained("responsibility-framing/predict-perception-bert-blame-assassin")

In [ ]:
irst results
pipe1 = pipeline("text-classification", model=model1, tokenizer=tokenizer1, max_length=512 )



**Use Model 1 on dataset** (texts)


In [ ]:
s=[]
for i in datac['text']:
  a=pipe1(i)
  s.append(a)


In [ ]:
datac["ass_blame"]=s
datac.head()
datac.to_csv('datawscores.csv') 

**Model 1 on titles**



In [ ]:
datac=pd.read_csv("datac.csv")
a=[]
for i in datac['title']:
  s=pipe1(i)
  a.append(s)


FileNotFoundError: ignored

**Model 2**

**Using the second model: Blame on the victim**
https://huggingface.co/responsibility-framing/predict-perception-bert-blame-victim


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer2 = AutoTokenizer.from_pretrained("responsibility-framing/predict-perception-bert-blame-victim", max_length=512, truncation=True, padding=True)
model2= AutoModelForSequenceClassification.from_pretrained("responsibility-framing/predict-perception-bert-blame-victim")
pipe2 = pipeline("text-classification", model=model2, tokenizer=tokenizer2, max_length=512)
#pipe2(text)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
#at the article level
c=[]
for i in datac['text']:
  a=pipe2(i)
  c.append(a)

#create new column and save dataset
datac["vict_blame"]=c
datac.head()
datac.to_csv('datawscores.csv') 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


KeyboardInterrupt: ignored

**Use model 2 with titles**

In [ ]:
#at the title level
a=[]
for i in datac['title']:
  s=pipe2(i)
  a.append(s)

datac["tit_vb"]=a
#datac.to_csv('datawscores2.csv') 


FileNotFoundError: ignored

**Model 2 at the sentence level**

In [ ]:
b=[]
for i in datac['sentence']:
  s=pipe2(i)
  b.append(s)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
datac["sentence_vb"]=b

datac.to_csv('labelcomplete.csv') 

**at the paragraph level**

In [ ]:
datasent=pd.read_csv("/content/paragraphs.csv")
p=[]
for i in datasent['paragraph']:
  s=pipe2(i)
  p.append(s)

datasent["para_vb"]=p

datasent.to_csv('npwscores.csv') 

**Try all the models together (to select the highest score)**

At the article level

In [ ]:
#blame on the assassin
tokenizer1 = AutoTokenizer.from_pretrained("responsibility-framing/predict-perception-bert-blame-assassin", max_length=512, truncation=True, padding=True)
model1 = AutoModelForSequenceClassification.from_pretrained("responsibility-framing/predict-perception-bert-blame-assassin")
pipe1 = pipeline("text-classification", model=model1, tokenizer=tokenizer1, max_length=512)

#blame on the victim
tokenizer2 = AutoTokenizer.from_pretrained("responsibility-framing/predict-perception-bert-blame-victim", max_length=512, truncation=True, padding=True)
model2= AutoModelForSequenceClassification.from_pretrained("responsibility-framing/predict-perception-bert-blame-victim")
pipe2 = pipeline("text-classification", model=model2, tokenizer=tokenizer2, max_length=512)

tokenizer3 = AutoTokenizer.from_pretrained("responsibility-framing/predict-perception-bert-blame-object", max_length=512, truncation=True, padding=True)
model3 = AutoModelForSequenceClassification.from_pretrained("responsibility-framing/predict-perception-bert-blame-object")
pipe3= pipeline("text-classification", model=model3, tokenizer=tokenizer3, max_length=512)

tokenizer4 = AutoTokenizer.from_pretrained("responsibility-framing/predict-perception-bert-blame-concept", max_length=512, truncation=True, padding=True)
model4= AutoModelForSequenceClassification.from_pretrained("responsibility-framing/predict-perception-bert-blame-concept")
pipe4= pipeline("text-classification", model=model4, tokenizer=tokenizer4, max_length=512)

tokenizer5 = AutoTokenizer.from_pretrained("responsibility-framing/predict-perception-bert-blame-none", max_length=512, truncation=True, padding=True)
model5 = AutoModelForSequenceClassification.from_pretrained("responsibility-framing/predict-perception-bert-blame-none")
pipe5= pipeline("text-classification", model=model5, tokenizer=tokenizer5, max_length=512)

"blame-assassin": AutoModelForSequenceClassification.from_pretrained(f"responsibility-framing/predict-perception-bert-blame-assassin"),
        "blame-victim": AutoModelForSequenceClassification.from_pretrained(f"responsibility-framing/predict-perception-bert-blame-victim"),
        "blame-object": AutoModelForSequenceClassification.from_pretrained(f"responsibility-framing/predict-perception-bert-blame-object"),
        "blame-concept": AutoModelForSequenceClassification.from_pretrained(f"responsibility-framing/predict-perception-bert-blame-concept"),
        "blame-none": AutoModelForSequenceClassification.from_pretrained(f"responsibility-framing/predict-perception-bert-blame-none"),


In [ ]:
datac=pd.read_csv("datac.csv")

In [ ]:
ab=[]
vb=[]
ob=[]
cb=[]
nb=[]

for i in datac['text']:
  a=pipe1(i)
  ab.append(a)
  v=pipe2(i)
  vb.append(v)
  o=pipe3(i)
  ob.append(o)
  c=pipe4(i)
  cb.append(c)
  n=pipe5(i)
  nb.append(n)

  

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

In [ ]:
datac["text_vb"]=vb
datac["text_ab"]=ab
datac["text_cb"]=cb
datac["text_ob"]=ob
datac["text_nb"]=nb

datac.to_csv('datawscores3.csv') 


In [ ]:
datac.head()

,victim,ID,Anno,date,Regione,Provincia,Città,Nome,Cognome,Note.di.genere,...,gender.j,picture,content,text,X,tit,text_vb,text_ab,text_cb,text_ob
0,sharon sapia barni,2,2021,2021-01-11,Lombardia,Bergamo,Cabiate,Sharon,Sapia Barni,(F),...,NaN,1.0,killer,E' un 25enne di Lentate sul Seveso l'uomo fin...,NaN,"Bambina morta a Cabiate, in carcere un 25enne ...","[{'label': 'LABEL_0', 'score': 0.4994814991950...","[{'label': 'LABEL_0', 'score': 0.6075556278228...","[{'label': 'LABEL_0', 'score': 0.5551394820213...","[{'label': 'LABEL_0', 'score': 0.5352218151092..."
1,sharon sapia barni,2,2021,2021-01-11,Lombardia,Bergamo,Cabiate,Sharon,Sapia Barni,(F),...,0.0,NaN,"carabinieri, house",Una vicenda choc che oggi ha avuto la sua svol...,NaN,Cabiate-choc: i Carabinieri arrestano il compa...,"[{'label': 'LABEL_0', 'score': 0.5181249380111...","[{'label': 'LABEL_0', 'score': 0.3396634459495...","[{'label': 'LABEL_0', 'score': 0.6494340896606...","[{'label': 'LABEL_0', 'score': 0.6325847506523..."
2,sharon sapia barni,2,2021,2021-01-11,Lombardia,Bergamo,Cabiate,Sharon,Sapia Barni,(F),...,NaN,1.0,carabinieri,"L'uomo arrestato a Como, il medico legale ha f...",NaN,"Violenta e picchia figlia della compagna, muor...","[{'label': 'LABEL_0', 'score': 0.5076031088829...","[{'label': 'LABEL_0', 'score': 0.5288490056991...","[{'label': 'LABEL_0', 'score': 0.6091281175613...","[{'label': 'LABEL_0', 'score': 0.5763741135597..."
3,sharon sapia barni,2,2021,2021-01-11,Lombardia,Bergamo,Cabiate,Sharon,Sapia Barni,(F),...,1.0,1.0,killer,segni di abusi sessuali sul corpo della piccol...,NaN,"""Como, botte e abusi sulla piccola Sharon prim...","[{'label': 'LABEL_0', 'score': 0.5093303918838...","[{'label': 'LABEL_0', 'score': 0.5822631120681...","[{'label': 'LABEL_0', 'score': 0.5409290790557...","[{'label': 'LABEL_0', 'score': 0.5800850987434..."
4,sharon sapia barni,2,2021,2021-01-11,Lombardia,Bergamo,Cabiate,Sharon,Sapia Barni,(F),...,NaN,1.0,victim,"«Sì, ho abusato di lei poi l’ho picchiata fino...",NaN,"""Ho ucciso Sharon a botte"". L\'assassino ha co...","[{'label': 'LABEL_0', 'score': 0.4434866309165...","[{'label': 'LABEL_0', 'score': 0.5890243649482...","[{'label': 'LABEL_0', 'score': 0.5586034655570...","[{'label': 'LABEL_0', 'score': 0.4517426490783..."


**Test extreme sentences**


In [ ]:
pipe1("é stata tutta colpa della moglie. Non voleva più avere rapporti con il marito e lui moriva di gelosia. Lui non ne poteva più e per questo giustamente in un raptus l'ha accoltellata. Lei se lo meritava. Lui la amava troppo ")

[{'label': 'LABEL_0', 'score': 0.7621191143989563}]

In [ ]:
pipe5("é stata tutta colpa della moglie. Non voleva più avere rapporti con il marito e lui moriva di gelosia. Lui non ne poteva più e per questo giustamente in un raptus l'ha accoltellata. Lei se lo meritava. Lui la amava troppo ")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'label': 'LABEL_0', 'score': 0.2412944734096527}]

In [ ]:
s="La donna accoltellata a Palermo. Il provvedimento di fermo contesta al marito le aggravanti dei “futili motivi” e della “crudeltà”. L'uomo aveva preparato una valigia. L’appello del procuratore aggiunto di Palermo Laura Vaccaro: “Donne, non abbiate paura di denunciare”. Si dispera papà Carlo: Povera figlia mia, aveva concesso a quell'uomo di restare a casa per amore dei figli. Ma di fatto erano ormai separati da mesi. Però lui continuava a ossessionarla, la controllava sempre. Ieri, Piera Napoli, 32enne cantante neomeolodica, è stata uccisa dal marito, Salvatore Baglione, 37 anni, dipendente di una macelleria. Lui, davanti al sostituto procuratore Federica Paiola, ha detto: Da quattro mesi c'erano discussioni. Ha parlato di raptus e di gelosia, tesi che la procura si appresta a contestare con l'aggravante della premeditazione. Ieri, mi aveva detto di non amarmi più - ha messo a verbale ieri pomeriggio, in una caserma dei carabinieri - l'ho raggiunta in bagno accoltellandola. Erano le 9,30, ho coperto il cadavere perchè non volevo vederlo. E perchè non lo vedessero i miei figli, che stavano dormendo. Mi sono sciacquato e cambiato. Ho accompagnato i bambini dalla nonna. Alle 12.40, Salvatore Baglione si è presentato alla stazione dei carabinieri dell'Uditore. L ho uccisa, ha detto. I militari sono subiti corsi a casa con un ambulanza, speravano ancora che la donna si potesse salvare. Ma non c era più nulla da dare. Il corpo di Piera Napoli era in bagno, in una pozza di sangue. Piera era ormai esasperata. Lui pensava di potere possedere la vita di sua moglie, sussurra Andrea Vicari, il marito della sorella di Piera Napoli. Un mese fa, in via Vanvitelli era arrivata anche una volante della polizia, chiamata dalla donna dopo l ennesima discussione. Ma alla fine lei non aveva voluto fare denuncia, racconta una vicina di casa. Chissà, forse, si poteva salvare Piera. Però, poi, dopo quella lite, sembrava che avesse fatto pace col marito. Ora, la procura contesta l omicidio volontario, con le aggravanti dei futili motivi, segnatamente per gelosia e della crudeltà. Scrive il pubblico ministero: Ha inflitto numerose e ripetute coltellate sul viso, sul capo e nel tronco, mediante un coltello che ha una lama di 20 centimetri. Il provvedimento di fermo parla di pericolo di fuga di Baglione che ha dimostrato - scrive il magistrato -  una particolare aggressività ed una allarmante determinazione nell efferato crimine, decidendo di presentarsi ai militari e denunciare l omicidio soltanto a distanza di alcune ore dal fatto, senza allertare prima la polizia, né chiedere nell immediatezza soccorso, quindi non facendosi neppure il minimo scrupolo di tentare di salvare la vita della moglie. Preoccupandosi invece in primo luogo di ripulirsi dalle tracce del reato e sistemare la proprie cose"

In [ ]:
s="«Sì, ho abusato di lei poi l’ho picchiata fino ad ucciderla». È scioccante la confessione di Gabriel Robert Marincat, il giovane in cella con l’accusa di omicidio volontario aggravato per aver ucciso la piccola Sharon. Questa mattina, davanti al pubblico ministero Antonia Pavan, il giovane - compagno e convivente della madre della bimba - ha deciso di confessare ogni cosa: la violenza sessuale sulla bimba e poi le botte mortali. Inizialmente arrestato con l’accusa di morte come conseguenza di maltrattamenti in famiglia (un reato che prevede pene fino a 24 anni) e violenza sessuale aggravata, ora il giovane deve rispondere di omicidio volontario. Un omicidio nato come epilogo di una giornata di violenze gravissime ai danni della piccola Sharon Barni, 18 mesi soltanto, che la mamma Silvia aveva affidato alle cure del suo convivente. La tragedia risale all’11 gennaio scorso. Marincat è nella casa di via Dante, a Cabiate, da solo. Con lui la figlia della sua compagna. La versione di quella maledetta giornata raccontata fino all’altroieri da Marincat recitava più o meno così: attorno alle 15.30/15.45 Sharon, giocando, tirava per sbaglio il cavo elettrico di una stufetta che si trovava sopra una scarpiera. L’elettrodomestico cadendo la colpisce alla testa. Lei piange per un po, ma poi si rimette a giocare. Un’ora più tardi si addormenta. Soltanto poco prima delle 19 viene chiamato il 118: l’équipe dell’elisoccorso cerca di salvare la bimba, ma invano. Arriverà all’ospedale Papa Giovanni XXIII di Bergamo già morta. Ieri l’uomo ha invece confessato e ammesso che la ricostruzione dell’accusa - fatta sulla base dell’indagine svolta dai carabinieri di Mariano Comense e della compagnia di Cantù e sulla ricostruzione del medico legale - è quella corretta: a bimba, quel giorno, ha subito abusi sessuali. Quindi è stata strattonata, maltrattata, poi colpite più volte alla testa. Fino alla morte. Ora la Procura chiedere il giudizio immediato per Marincat."

In [ ]:
print(pipe1(s))
print(pipe2(s))
print(pipe3(s))
print(pipe4(s))
print(pipe5(s))


[{'label': 'LABEL_0', 'score': 0.6653290390968323}]
[{'label': 'LABEL_0', 'score': 0.52265465259552}]
[{'label': 'LABEL_0', 'score': 0.5991404056549072}]
[{'label': 'LABEL_0', 'score': 0.6100441217422485}]
[{'label': 'LABEL_0', 'score': 0.32664161920547485}]


In [ ]:
print(s)

La donna accoltellata a Palermo. Il provvedimento di fermo contesta al marito le aggravanti dei “futili motivi” e della “crudeltà”. L'uomo aveva preparato una valigia. L’appello del procuratore aggiunto di Palermo Laura Vaccaro: “Donne, non abbiate paura di denunciare”. Si dispera papà Carlo: Povera figlia mia, aveva concesso a quell'uomo di restare a casa per amore dei figli. Ma di fatto erano ormai separati da mesi. Però lui continuava a ossessionarla, la controllava sempre. Ieri, Piera Napoli, 32enne cantante neomeolodica, è stata uccisa dal marito, Salvatore Baglione, 37 anni, dipendente di una macelleria. Lui, davanti al sostituto procuratore Federica Paiola, ha detto: Da quattro mesi c'erano discussioni. Ha parlato di raptus e di gelosia, tesi che la procura si appresta a contestare con l'aggravante della premeditazione. Ieri, mi aveva detto di non amarmi più - ha messo a verbale ieri pomeriggio, in una caserma dei carabinieri - l'ho raggiunta in bagno accoltellandola. Erano le 9